In [1]:
import json
import tensorflow as tf
# from keras.preprocessing import image
from keras.models import Sequential
from keras.utils import image_dataset_from_directory
from keras.layers import InputLayer, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.metrics import Accuracy, Precision, Recall
import cv2 as cv
from matplotlib import pyplot as plt

In [2]:
classes: list[str] = []

with open('land_use_images\\label_map.json', 'rb') as f:
    classes = json.loads(f.read())

classes = [x for x in classes.keys()]
(image_width, image_height) = 256, 256

In [4]:
def normalizing_data(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

In [5]:
train_dataset: tf.data.Dataset = image_dataset_from_directory(
    "land_use_images\\images_train_test_val\\train",
    labels="inferred",
    label_mode="int",
    class_names=classes,
    color_mode="rgb",
    batch_size=50,
    image_size=(image_width, image_height),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True
)
train_dataset = train_dataset.map(normalizing_data)

Found 7350 files belonging to 21 classes.


In [6]:
test_dataset: tf.data.Dataset = image_dataset_from_directory(
    "land_use_images\\images_train_test_val\\test",
    labels="inferred",
    label_mode="int",
    class_names=classes,
    color_mode="rgb",
    batch_size=50,
    image_size=(image_width, image_height),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True
)
test_dataset = test_dataset.map(normalizing_data)

Found 1050 files belonging to 21 classes.


In [7]:
validation_dataset: tf.data.Dataset = image_dataset_from_directory(
    "land_use_images\\images_train_test_val\\validation",
    labels="inferred",
    label_mode="int",
    class_names=classes,
    color_mode="rgb",
    batch_size=50,
    image_size=(image_width, image_height),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True
)
validation_dataset = validation_dataset.map(normalizing_data)

Found 2100 files belonging to 21 classes.


In [8]:
input_shape = (image_width, image_height, 3)

In [9]:
model = Sequential()

#model.add(InputLayer(shape=input_shape))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=21, activation='softmax'))  # Supondo 10 classes para a saída

#model.add(Conv2D(128, 3, activation='relu', padding='same'))
#model.add(Conv2D(128, 3, activation='relu', padding='same'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Conv2D(258, 3, activation='relu', padding='same'))
#model.add(Conv2D(258, 3, activation='relu', padding='same'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Flatten())
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(rate=0.5))
#model.add(Dense(64, activation="relu")),
#model.add(Dropout(rate=0.5))
#model.add(Dense(21, activation='softmax'))

'''
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
'''



"\nmodel.add(Conv2D(32, (2, 2), input_shape=input_shape))\nmodel.add(Activation('relu'))\nmodel.add(MaxPooling2D(pool_size=(2, 2)))\n \nmodel.add(Conv2D(32, (2, 2)))\nmodel.add(Activation('relu'))\nmodel.add(MaxPooling2D(pool_size=(2, 2)))\n \nmodel.add(Conv2D(64, (2, 2)))\nmodel.add(Activation('relu'))\nmodel.add(MaxPooling2D(pool_size=(2, 2)))\n \nmodel.add(Flatten())\nmodel.add(Dense(64))\nmodel.add(Activation('relu'))\nmodel.add(Dropout(0.5))\nmodel.add(Dense(1))\nmodel.add(Activation('sigmoid'))\n"

In [10]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)

In [11]:
model.fit(train_dataset, epochs=5, steps_per_epoch=250, batch_size=32, validation_data=test_dataset)

Epoch 1/5
147/250 ━━━━━━━━━━━━━━━━━━━━ 2:41 2s/step - accuracy: 0.0633 - loss: 3.9202

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


250/250 ━━━━━━━━━━━━━━━━━━━━ 244s 957ms/step - accuracy: 0.0745 - loss: 3.6107 - val_accuracy: 0.1790 - val_loss: 2.6746
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 201s 797ms/step - accuracy: 0.2296 - loss: 2.5637 - val_accuracy: 0.2990 - val_loss: 2.2971
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 195s 774ms/step - accuracy: 0.4178 - loss: 1.9636 - val_accuracy: 0.3257 - val_loss: 2.1594
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 195s 777ms/step - accuracy: 0.6236 - loss: 1.2730 - val_accuracy: 0.3019 - val_loss: 2.7081
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 198s 788ms/step - accuracy: 0.7711 - loss: 0.7910 - val_accuracy: 0.3467 - val_loss: 2.5817


In [12]:
model.predict(validation_dataset)

42/42 ━━━━━━━━━━━━━━━━━━━━ 17s 372ms/step


array([[7.2651102e-10, 8.8237440e-03, 1.0483840e-15, ..., 1.3017108e-03,
        1.6214914e-07, 6.3771588e-01],
       [3.6846899e-04, 4.5482289e-02, 6.2448669e-02, ..., 3.6155707e-03,
        2.6788991e-03, 3.0292847e-04],
       [7.2188623e-04, 9.1799575e-06, 6.7550660e-05, ..., 9.7286184e-06,
        2.8064630e-06, 6.6404618e-06],
       ...,
       [6.1321293e-04, 4.1997889e-03, 3.9721904e-06, ..., 2.8608914e-03,
        1.8111410e-03, 1.6452713e-02],
       [3.3187727e-15, 7.1552146e-04, 2.8276863e-13, ..., 5.7776101e-02,
        8.8836165e-04, 2.2100368e-01],
       [6.3134104e-01, 4.2348489e-05, 7.4917452e-06, ..., 3.1513207e-02,
        6.7833878e-02, 1.1677042e-03]], dtype=float32)